2020-11-04 01:08:20,908 - kedro.io.data_catalog - INFO - Loading data from `trips` (CSVDataSet)...


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,11/17/2018 05:19:13 PM,11/17/2018 05:34:22 PM,5,3.04,1,N,236,166,1,13.0,0.0,0.5,3.45,0.0,0.3,17.25
1,2,11/17/2018 05:41:19 PM,11/17/2018 05:45:44 PM,5,0.92,1,N,151,166,1,5.0,0.0,0.5,1.16,0.0,0.3,6.96
2,2,11/17/2018 05:50:24 PM,11/17/2018 06:28:22 PM,5,6.14,1,N,151,186,1,28.5,0.0,0.5,5.55,0.0,0.3,34.85
3,2,11/17/2018 05:15:50 PM,11/17/2018 05:44:20 PM,5,4.09,1,N,48,237,1,19.5,0.0,0.5,4.06,0.0,0.3,24.36
4,2,11/17/2018 05:55:01 PM,11/17/2018 05:55:25 PM,5,0.11,1,N,236,237,2,2.5,0.0,0.5,0.00,0.0,0.3,3.30


In [19]:
import pandas as pd
import numpy as np

In [22]:
def extract_hour(row):
    return float(int(row['pickup_date'].hour) + int(row['pickup_date'].minute)/60)

def night_hour(row):
    return row['pickup_hour'] < 4

def weekday(row):
    return row['pickup_date'].weekday()

def weekday(row):
    return row['pickup_date'].weekday()

def is_weekend(row):
    return row['weekday']>4

def is_airport_destination(row):
    return row['weekday']>4



In [23]:
def preprocess(dataframe: pd.DataFrame) -> pd.DataFrame:
    dataframe['tpep_pickup_datetime'] = pd.to_datetime(dataframe['tpep_pickup_datetime'])
    dataframe.rename(columns={'tpep_pickup_datetime': 'pickup_date'}, inplace=True)
    dataframe['pickup_hour'] = dataframe.apply (lambda row: extract_hour(row), axis=1)

    dataframe['sin_pickup_hour'] = np.sin(dataframe['pickup_hour'])
    dataframe['cos_pickup_hour'] = np.cos(dataframe['pickup_hour'])
    dataframe['night_hours'] = dataframe.apply (lambda row: night_hour(row), axis=1)
    dataframe['weekday'] = dataframe.apply (lambda row: weekday(row), axis=1)
    dataframe['weekend'] = dataframe.apply (lambda row: is_weekend(row), axis=1)
    dataframe['passenger_count'] = dataframe['passenger_count'].astype(int)
    dataframe['label'] = dataframe.apply(lambda row: 1 if row['DOLocationID'] in [1, 132, 138] else 0, axis=1)
    dataframe = dataframe[[
        'pickup_hour',
        'sin_pickup_hour',
        'cos_pickup_hour',
        'night_hours',
        'weekday',
        'weekend',
        'passenger_count',
        'label'
    ]]

    df_pos = dataframe[dataframe.label.eq(1)]
    df_neg = dataframe[dataframe.label.eq(0)]
    df_neg = df_neg.head(1000)

    dataframe = pd.concat([df_pos, df_neg], ignore_index=True)

    return dataframe

In [24]:
df = catalog.load("trips")
df.head()

2020-11-04 01:17:13,563 - kedro.io.data_catalog - INFO - Loading data from `trips` (CSVDataSet)...


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,11/17/2018 05:19:13 PM,11/17/2018 05:34:22 PM,5,3.04,1,N,236,166,1,13.0,0.0,0.5,3.45,0.0,0.3,17.25
1,2,11/17/2018 05:41:19 PM,11/17/2018 05:45:44 PM,5,0.92,1,N,151,166,1,5.0,0.0,0.5,1.16,0.0,0.3,6.96
2,2,11/17/2018 05:50:24 PM,11/17/2018 06:28:22 PM,5,6.14,1,N,151,186,1,28.5,0.0,0.5,5.55,0.0,0.3,34.85
3,2,11/17/2018 05:15:50 PM,11/17/2018 05:44:20 PM,5,4.09,1,N,48,237,1,19.5,0.0,0.5,4.06,0.0,0.3,24.36
4,2,11/17/2018 05:55:01 PM,11/17/2018 05:55:25 PM,5,0.11,1,N,236,237,2,2.5,0.0,0.5,0.00,0.0,0.3,3.30


In [25]:
output = preprocess(df)
output.head()

/home/dwarszawski/virtualenvs/demo/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,pickup_hour,sin_pickup_hour,cos_pickup_hour,night_hours,weekday,weekend,passenger_count,label
0,17.650000,-0.931878,0.362772,False,5,True,5,1
1,17.633333,-0.937795,0.347191,False,5,True,1,1
2,17.416667,-0.990506,0.137470,False,5,True,2,1
3,17.583333,-0.953975,0.299887,False,5,True,3,1
4,17.133333,-0.989444,-0.144914,False,5,True,1,1


In [27]:
catalog.save("preprocessed_trips", output)

2020-11-04 01:19:15,945 - kedro.io.data_catalog - INFO - Saving data to `preprocessed_trips` (CSVDataSet)...
